In [1]:
import pandas as pd
import os
from IPython.display import HTML

best_brawlers = 12

df = pd.read_csv('brawlers.csv')

def clean_brawlers_string(s):
    try:
        return s.replace('\\n',' ')
    except:
        return s

def clean_brawler_name(brawler):
    if brawler=='8-Bit':
        return brawler
    elif brawler=='Mr.P':
        return 'Mr. P'
    elif 'Colonel' in brawler or 'Ruffs' in brawler:
        return 'Ruffs'
    else:
        return brawler.replace('-',' ')
    
def clean_string(s):
    try:
        s = s.split(', ')
        s = [(x.replace('%','')) for x in s]
        return ', '.join(s)
    except:
        return s

def round_string(s):
    try:
        s = s.split(', ')
        s = [str(int(float(x))) for x in s]
        return ', '.join(s)
    except:
        return s
    
def nth_repl(s, sub, repl, n):
    try:
        find = s.find(sub)
        # If find is not -1 we have found at least one match for the substring
        i = find != -1
        # loop util we find the nth or we find no match
        while find != -1 and i != n:
            # find + 1 means we start searching from after the last match
            find = s.find(sub, find + 1)
            i += 1
        # If i is equal to n we found nth match so replace
        if i == n:
            return s[:find] + repl + s[find+len(sub):]
        return s
    except:
        return s

images_dir = './brawler_images'
images_path = os.listdir(images_dir)
gamemodes_dir = './gamemode_images'

df['best_brawlers'] = df['best_brawlers'].map(clean_brawlers_string)

images = {}
for image in images_path:
    images[clean_brawler_name(image.replace('.png',''))] = os.path.join(images_dir, image)
    
list_of_gamemodes = df['gamemodes']

for gamemode in list_of_gamemodes:
    fname = gamemode.replace(' ','-')+'.png'
    images[gamemode] = os.path.join(gamemodes_dir, fname)
    
columns = [str(x) for x in list(range(1,best_brawlers+1))]
    
df[columns] = df['best_brawlers'].str.split(', ', best_brawlers-1, expand=True)

def path_to_image_html(path):
    return '<img src="'+ path + '" width="80" >'
    
def path_to_image_html_smaller(path):
    return '<img src="'+ path + '" width="40" >'

format_dict = {}
for image_col in columns+['gamemodes']:
    if image_col=='gamemodes':
        format_dict[image_col] = path_to_image_html_smaller
    else:
        format_dict[image_col] = path_to_image_html
        
for col in columns+['gamemodes']:
    df[col] = df[col].map(images)

df = df.drop(['best_brawlers'],axis=1)
df['win_rates'] = df['win_rates'].map(clean_string)
df['win_rates'] = df['win_rates'].map(round_string)
df['win_rates'] = df['win_rates'].map(lambda s: nth_repl(s, ", ","<br>",best_brawlers/3))
df['use_rates'] = df['use_rates'].map(clean_string)
df['use_rates'] = df['use_rates'].map(lambda s: nth_repl(s, ", ","<br>",best_brawlers/3))
df = df.rename(columns={"num_brawlers": "n"})
df = df[['gamemodes','map']+columns+['win_rates','use_rates','n','filter']]
df.index += 1
# df.to_html('best_brawlers.html', escape=False, formatters=format_dict, render_links=True)

In [2]:
df2 = df.fillna('   ').head(9)
HTML(df2.to_html(escape=False, formatters=format_dict))

,gamemodes,map,1,2,3,4,5,6,7,8,9,10,11,12,win_rates,use_rates,n,filter
1,,Shooting Star,,,,,,,,,,,,,"69, 67, 67, 6565, 64, 62, 62, 62, 62, 61, 60","1.2, 4.5, 6.5, 1.23.7, 4.5, 1.1, 2.0, 2.4, 5.7, 8.5, 2.8",58,
2,,Canal Grande,,,,,,,,,,,,,"64, 64, 63, 6262, 61, 61, 60, 59, 59, 58, 58","1.3, 2.7, 1.7, 2.43.3, 1.3, 1.6, 1.2, 2.9, 1.7, 1.3, 3.5",59,600+
3,,Layer Cake,,,,,,,,,,,,,"63, 60, 59, 5958, 57, 57, 57, 57, 57, 56, 56","0.9, 1.5, 2.2, 5.75.6, 2.3, 4.4, 1.3, 2.7, 0.9, 5.7, 2.7",56,
4,,Super Beach,,,,,,,,,,,,,"63, 62, 58, 5857, 57, 56, 56, 55, 54, 54, 54","1.4, 2.8, 3.7, 1.93.7, 2.2, 1.9, 3.6, 3.7, 1.4, 2.8, 5.6",59,600+
5,,Pinhole Punt,,,,,,,,,,,,,"79, 71, 61, 6160, 59, 59, 59, 57, 56, 56, 56","0.9, 4.2, 15.6, 1.33.9, 2.5, 1.1, 2.2, 1.7, 3.0, 1.1, 2.2",59,600+
6,,Sneaky Fields,,,,,,,,,,,,,"61, 61, 59, 5858, 57, 56, 56, 55, 55, 54, 53","3.1, 3.3, 2.2, 4.41.5, 3.3, 1.1, 1.7, 2.8, 1.1, 5.0, 2.8",59,600+
7,,Hard Rock Mine,,,,,,,,,,,,,"61, 59, 57, 5756, 56, 56, 56, 55, 55, 55, 55","1.3, 1.1, 1.7, 4.61.7, 1.2, 3.8, 1.6, 1.9, 2.2, 1.5, 3.4",59,600+
8,,Double Swoosh,,,,,,,,,,,,,"63, 61, 61, 6059, 58, 58, 58, 56, 55, 55, 54","2.0, 4.5, 1.8, 5.24.9, 1.1, 4.4, 1.6, 6.4, 1.8, 4.2, 2.9",53,
9,,Crystal Arcade,,,,,,,,,,,,,"61, 61, 56, 5655, 55, 55, 55, 55, 54, 54, 54","1.9, 1.2, 2.2, 8.34.1, 3.9, 2.1, 2.1, 2.2, 1.0, 3.8, 1.2",58,600+


In [3]:
df3 = df.fillna('   ').tail(len(df)-9)
HTML(df3.to_html(escape=False, formatters=format_dict))

,gamemodes,map,1,2,3,4,5,6,7,8,9,10,11,12,win_rates,use_rates,n,filter
10,,Pit Stop,,,,,,,,,,,,,"66, 63, 63, 6362, 60, 60, 59, 58, 58, 57, 55","0.9, 1.4, 2.9, 4.510.6, 7.4, 2.1, 9.0, 9.4, 2.5, 1.6, 9.1",57,600+
11,,Safe Zone,,,,,,,,,,,,,"67, 63, 59, 5958, 57, 57, 57, 56, 56, 55, 54","1.3, 2.0, 1.2, 1.55.5, 1.5, 2.9, 5.3, 2.0, 2.1, 3.0, 1.3",58,600+
12,,Bridge Too Far,,,,,,,,,,,,,"60, 59, 59, 5959, 58, 57, 56, 55, 55, 55, 54","4.1, 4.9, 1.7, 4.11.0, 3.1, 1.1, 0.9, 1.4, 1.3, 16.5, 10.7",57,600+
13,,Ring of Fire,,,,,,,,,,,,,"70, 64, 64, 6262, 61, 60, 59, 59, 58, 58, 58","1.5, 12.9, 1.4, 1.52.8, 1.3, 1.0, 2.3, 1.6, 0.9, 3.7, 1.5",58,600+
14,,Dueling Beetles,,,,,,,,,,,,,"78, 73, 71, 7171, 71, 70, 70, 69, 69, 68, 68","7.2, 4.8, 5.6, 1.32.4, 8.7, 2.8, 1.4, 1.5, 1.0, 2.2, 4.0",59,
15,,Open Zone,,,,,,,,,,,,,"68, 67, 67, 6665, 63, 62, 62, 61, 60, 59, 59","1.0, 3.1, 2.0, 13.33.0, 3.5, 1.1, 1.6, 2.7, 5.3, 2.5, 7.2",59,600+
16,,Goldarm Gulch,,,,,,,,,,,,,"73, 72, 70, 6966, 63, 63, 61, 60, 59, 58, 58","1.9, 2.4, 1.0, 1.93.7, 5.2, 4.6, 1.9, 1.9, 3.7, 1.7, 0.9",58,600+
17,,Out in the Open,,,,,,,,,,,,,"60, 60, 59, 5958, 58, 57, 57, 57, 56, 56, 56","1.7, 4.5, 3.5, 3.02.1, 4.6, 1.1, 1.5, 1.7, 1.4, 3.3, 1.2",58,600+
18,,Belles Rock,,,,,,,,,,,,,"61, 59, 59, 5958, 58, 57, 57, 57, 56, 56, 56","2.3, 2.1, 2.2, 1.14.9, 5.4, 4.2, 2.8, 1.5, 1.2, 5.6, 3.1",59,600+


In [4]:
import itertools

df = pd.read_csv('brawlers.csv')[['best_brawlers']]
df['best_brawlers'] = df['best_brawlers'].map(clean_brawlers_string)

brawlers = {}
brawlers_freq = {}
for i in range(len(df)):
    brawlers_list = df.iloc[i]['best_brawlers'].split(', ')
    for j in range(len(brawlers_list)):
        if brawlers_list[j] not in brawlers:
            brawlers[brawlers_list[j]] = len(brawlers_list)-j 
            brawlers_freq[brawlers_list[j]] = 1
        else:
            brawlers[brawlers_list[j]] += len(brawlers_list)-j 
            brawlers_freq[brawlers_list[j]]+=1
            
def path_to_image_html_smallest(path):
    return '<img src="'+ path + '" width="30" >'
    
def print_dataframe(df):
    df.columns = ['brawlers','score']
    df = df.sort_values(by=['score'],ascending=False).reset_index(drop=True)
    df.index+=1
    df['brawlers'] = df['brawlers'].map(lambda x: images[x])
    return df

df = pd.DataFrame.from_dict([brawlers_freq]).T.reset_index()
df = print_dataframe(df)
df.columns = ['brawlers','frequency']

df2 = pd.DataFrame.from_dict([brawlers]).T.reset_index()
df2 = print_dataframe(df2)

def get_best_maps():
    df = pd.read_csv('brawlers.csv')[['map','best_brawlers','win_rates']]
    df['best_brawlers'] = df['best_brawlers'].map(clean_brawlers_string)
    df['best_brawlers'] = df['best_brawlers'].str.split(', ')
    df['win_rates'] = df['win_rates'].str.split(', ')
    brawler_set = sorted(list(set(list(itertools.chain(*df['best_brawlers'])))))

    brawler_map_dict = {}
    for brawler in brawler_set:
        best_maps = []
        for i in range(len(df)):
            best_brawlers = df.iloc[i]['best_brawlers']
            win_rates = df.iloc[i]['win_rates']
            mapname = df.iloc[i]['map']
            if brawler in best_brawlers:
                best_maps.append([mapname, win_rates[best_brawlers.index(brawler)]])
        brawler_map_dict[brawler] = best_maps

    brawler_map_dict
    df = pd.DataFrame(brawler_set)
    df.columns = ['brawlers']
    df['best_maps'] = df['brawlers'].map(lambda x:brawler_map_dict[x])
    df['best_maps'] = df['best_maps'].map(lambda x: sorted(x, key=lambda y:y[1])[::-1][:3])
    df['best_maps'] = df['best_maps'].map(lambda x: ', '.join([y[0] for y in x]))
    df['brawlers'] = df['brawlers'].map(lambda x: images[x])
    return df

best_maps = get_best_maps()

df3 = df.merge(df2, on=['brawlers']).head(12)
df4 = df3.merge(best_maps, on=['brawlers'])

columns = ['best_map1', 'best_map2','best_map3']
df4[columns] = df4['best_maps'].str.split(', ', expand=True)
df4 = df4.drop(['best_maps'],axis=1)

def get_gamemode_dict():
    df = pd.read_csv('brawlers.csv')[['gamemodes','map']]
    return dict(zip(df['map'], df['gamemodes']))

gamemode_dict = get_gamemode_dict()

df4['mode1'] = df4['best_map1'].map(lambda x: images[gamemode_dict[x]])
df4['mode2'] = df4['best_map2'].map(lambda x: images[gamemode_dict[x]])
df4['mode3'] = df4['best_map3'].map(lambda x: images[gamemode_dict[x]])

format_dict['brawlers'] = path_to_image_html_smallest
format_dict[''] = path_to_image_html_smallest

df4 = df4[['brawlers','frequency','score','mode1','best_map1','mode2','best_map2','mode3','best_map3']]
df4.columns = ['brawlers','freq','weighted_score','','bestmap1','','bestmap2','','bestmap3']
df4 = df4.sort_values(by=['weighted_score', 'freq'], ascending=False).reset_index(drop=True)
df4.index+=1
HTML(df4.to_html(escape=False, formatters=format_dict))

,brawlers,freq,weighted_score,,bestmap1,,bestmap2,,bestmap3
1,,12,122,,Dueling Beetles,,Goldarm Gulch,,Open Zone
2,,7,75,,Dueling Beetles,,Shooting Star,,Goldarm Gulch
3,,7,69,,Ring of Fire,,Pit Stop,,Super Beach
4,,7,65,,Canal Grande,,Hard Rock Mine,,Crystal Arcade
5,,9,49,,Goldarm Gulch,,Double Swoosh,,Out in the Open
6,,7,49,,Goldarm Gulch,,Open Zone,,Ring of Fire
7,,8,46,,Dueling Beetles,,Canal Grande,,Open Zone
8,,7,36,,Dueling Beetles,,Safe Zone,,Belles Rock
9,,7,35,,Dueling Beetles,,Pit Stop,,Pinhole Punt
10,,6,35,,Open Zone,,Goldarm Gulch,,Pinhole Punt


In [5]:
players = pd.read_csv('../output/c9aurac_brawler_levels.csv')
team_tags = ['#C9LR0R0V','#UCY09URC','#2YQUPUYJ'] #C9 - Silver, Blue, Hogg
# team_tags = ['#V8VRPRYQ','#89GV9UG9Q','#C29RQJLU'] # C6 Tribe

# team_tags = ['#2PR80P8CU','#J2RLUJP2','#LQPP9Y2R8']
players = players[players['tag'].isin(team_tags)][['player','level_11s','brawlers_11']].reset_index(drop=True)

df5 = df4.copy()[['brawlers','freq','weighted_score']].reset_index(drop=True)
df5['names'] = df5['brawlers'].map(lambda x: list(images.keys())[list(images.values()).index(x)])
df5['names'] = df5['names'].map(lambda x:x.upper())
df5 = df5.set_index('names')
player_list = list(players['player'])
for player_name in player_list:
    df5[player_name]=''
    brawler_list = list(players[players['player']==player_name]['brawlers_11'])[0].split(', ')
    for brawler in brawler_list:
        if brawler in df5.index:
            df5.at[brawler, player_name]=1
df5 = df5.reset_index(drop=True)
df5.index+=1
HTML(df5.to_html(escape=False, formatters=format_dict))

,brawlers,freq,weighted_score,Blue,Hogglific,ZSilverZ
1,,12,122,,1,1
2,,7,75,1,1,
3,,7,69,,,1
4,,7,65,,,1
5,,9,49,,1,1
6,,7,49,,1,
7,,8,46,,1,1
8,,7,36,1,1,1
9,,7,35,1,,
10,,6,35,,,
